In [ ]:
from ExpenseHandler import ExpenseHandler

import plotly.graph_objects as go

In [ ]:
# Read CSV file

expenses_obj = ExpenseHandler()

expenses_obj.fillZeroExpenseDates()
expenses_obj.fillMonthNumber()

expenses_df = expenses_obj.getExpenseDF()

In [ ]:
# Time series plot (per day)
costs_df = expenses_obj.annualCosts('date')

# Begin plotting here
fig = go.Figure([
    go.Scatter(
        x=costs_df['date'], 
        y=costs_df['cost'],
        mode='lines+markers'
    )
])

fig.update_layout(title_text='Overview of expenses (Daily)')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
# Time series plot (per month)
costs_df = expenses_obj.annualCosts('month')

# Begin plotting here
fig = go.Figure([
    go.Scatter(
        x=costs_df['month'], 
        y=costs_df['cost'],
        mode='lines+markers'
    )
])

fig.update_layout(title_text='Overview of expenses (Monthly)')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
categories, values = expenses_obj.categoryCounts()

fig = go.Figure([go.Bar(x=categories, y=values)])

fig.update_layout(
    title='Count categories of expenses',
    yaxis=dict(
        title='Counts'
    ),
    xaxis=dict(
        title='Categories'
    )
)

fig.show()

In [ ]:
expenses_only_df = expenses_df[expenses_df['category'] != 'zero expenses']
non_expenses_df = expenses_df[expenses_df['category'] == 'zero expenses']

fig = go.Figure([go.Bar(x=['Expenses','Zero expenses'], y=[len(expenses_only_df),len(non_expenses_df)])])

fig.update_layout(
    title='Counts: Expenses vs Zero Expenses',
    yaxis=dict(
        title='Counts'
    ),
    xaxis=dict(
        title='Categories'
    ),
    width=600,
    height=600
)

fig.show()

In [ ]:
# Find day of week from date
day_number_week = [date.weekday() for date in expenses_df['date']]

expenses_df = expenses_df.assign(day = day_number_week)
print(expenses_df.head(20))

In [ ]:
days_df = expenses_df.groupby('day')['cost'].sum()

days = [calendar.day_name[key] for key in days_df.keys()]
total_spent = days_df.values

fig = go.Figure([go.Bar(x=days, y=total_spent)])

fig.update_layout(
    title='Total spent: week days calculation',
    yaxis=dict(
        title='Total'
    ),
    xaxis=dict(
        title='Day'
    ),
    width=800,
    height=600
)

fig.show()